In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import math
from collections import Counter
from IPython.display import display_html
from sklearn.neural_network import MLPClassifier
from imblearn.under_sampling import RandomUnderSampler

In [2]:
events_called = 100

### Importing data

In [3]:
def func_import_100_sample(events_called):
    hits = []
    cells = []
    particles = []
    truth = []

    # append datasets into the list
    for i in range(0,events_called):
        if i < 10:
            temp_df = pd.read_csv('train_100_events/event00000100%d-hits.csv' % i)
            #c_c = pd.read_csv('train_100_events/event00000100%d-cells.csv'% i )
            p_c = pd.read_csv('train_100_events/event00000100%d-particles.csv'% i)
            t_c = pd.read_csv('train_100_events/event00000100%d-truth.csv'% i)
            hits.append(temp_df)
            #cells.append(c_c)
            particles.append(p_c)
            truth.append(t_c)
        if i >= 10:
            temp_df = pd.read_csv('train_100_events/event0000010%d-hits.csv' % i)
            #c_c = pd.read_csv('train_100_events/event0000010%d-cells.csv'% i )
            p_c = pd.read_csv('train_100_events/event0000010%d-particles.csv'% i)
            t_c = pd.read_csv('train_100_events/event0000010%d-truth.csv'% i)
            #cells.append(c_c)
            hits.append(temp_df)
            particles.append(p_c)
            truth.append(t_c)
    return cells , hits , particles , truth

In [4]:
%%time
cells_all , hits_all , particles_all , truth_all = func_import_100_sample(events_called)
data_detectors = pd.read_csv(r"detectors.csv")


Wall time: 15.3 s


In [5]:
len(hits_all)

100

### Making functions

In [6]:
def func_cleaning_data( hits, particles , truth, pt_cut_start, pt_cut_end):
    #Finding all hit_id that is noice, to use in other files for removing nocie in them.
    def noice(truth):
        truth_hit_id_noice = [truth.hit_id[i] for i in range(len(truth)) if truth.particle_id[i] == 0]
        return truth_hit_id_noice
    truth_hit_id_noice_list = noice(truth)
    #Removing all the noice in the truth file:
    truth_zero_noice = truth.drop(truth.index[truth['particle_id'] == 0]).reset_index()
    #Removing all the data where the nhits is >=3 :
    particles_zero_noice = particles.drop(particles.index[particles['nhits'] <= 3]).reset_index()
    #Sorting the particles
    particles_zero_noice_sorted_unique = particles_zero_noice.sort_values(by = "particle_id",ascending=True)
    #Making a function that can remove all row that has a value in a list
    def FRBV(file_name, column_name, list_of_values):
        return file_name[~file_name[column_name].isin(list_of_values)]
    # #Removing all the noice in the cells file:
    # cells_zero_noice = FRBV(cells , "hit_id" , truth_hit_id_noice_list).reset_index()
    #Removing all the noice in the hits file:
    hits_zero_noice = FRBV(hits , "hit_id" , truth_hit_id_noice_list).reset_index()
    #Making a function that can remove all the data, that has a nhits over 7.
    def nhit_over_7(data):
        data = [data.particle_id[i] for i in range(len(data)) if data.nhits[i] > 7]
        return data
    #Removing all the data where nhits is less then 7
    particle_id_with_nhits_over_7 = nhit_over_7(particles_zero_noice_sorted_unique)
    #Removing all the data where nhits is over then 7
    particle_id_with_nhits_lees_7 = FRBV(particles_zero_noice_sorted_unique , "particle_id" , particle_id_with_nhits_over_7).drop("index",axis = 1).reset_index().drop("index",axis = 1)
    #Removing all the data where a particle_id has more then 7 nhits.
    truth_zero_noice_nhits_lees_7 = FRBV(truth_zero_noice , "particle_id" , particle_id_with_nhits_over_7).drop("index",axis = 1).reset_index().drop("index",axis = 1)
    #Making a function that can make a list of the hit_ids that has a weight of 0.
    def weight_equle_0(data):
        data = [data.hit_id[i] for i in range(len(data)) if data.weight[i] == 0]
        return data
    #Using the weight_equle_0 function to make a list of hit_id´s that has a weight equle 0
    truth_weight_0_list = weight_equle_0(truth_zero_noice_nhits_lees_7)
    #Using the list of hit_id´s that has a weight equle 0, to remove the rows in truth that has that hit_id.
    truth_zero_noice_nhits_lees_7_weight_0 = FRBV(truth_zero_noice_nhits_lees_7,"hit_id",truth_weight_0_list).reset_index().drop("index",axis = 1)
    
    #Removing the data where the particle_id has less then 7 nhits.
    truth_zero_noice_nhits_over_7 = FRBV(truth_zero_noice , "particle_id" ,truth_zero_noice_nhits_lees_7.particle_id).drop("index",axis = 1).reset_index().drop("index",axis = 1)
    #Removing the data where the hit_id has less the 7 nhits.
    hits_zero_noice_nhits_lees_7 = FRBV(hits_zero_noice, "hit_id",truth_zero_noice_nhits_over_7["hit_id"]).drop("index",axis = 1).reset_index().drop("index",axis = 1)
    #Making a list of all the particle_ids that has over 3 nhits
    particles_id_over_3 = [particles.particle_id[i] for i in range(len(particles)) if particles.nhits[i] > 3]
    #Removing all the data in truth that has less the 3 nhits. Used for later in cells and hits.  
    truth_zero_noice_over_3 = FRBV(truth_zero_noice,"particle_id",particles_id_over_3).drop("index",axis = 1).reset_index().drop("index",axis = 1)
    #Removing all the data in hits that has over the 3 nhits.
    hits_zero_noice_nhits_lees_7_over_3_with_weight_0 = FRBV(hits_zero_noice_nhits_lees_7,"hit_id",truth_zero_noice_over_3.hit_id).reset_index().drop("index",axis = 1)
    #Making a list of hit_id that is not in truth but is in hits
    hit_id_in_hits_but_not_in_truth = FRBV(hits_zero_noice_nhits_lees_7_over_3_with_weight_0,"hit_id",truth_zero_noice_nhits_lees_7_weight_0.hit_id).reset_index().drop("index",axis = 1)
    #Using that hits not in truth, and then removing them from hits. 
    hits_zero_noice_nhits_lees_7_over_3_without_weight_0 = FRBV(hits_zero_noice_nhits_lees_7_over_3_with_weight_0,"hit_id",hit_id_in_hits_but_not_in_truth.hit_id).reset_index().drop("index",axis = 1)

#     #Removing the data where the hit_id has less the 7 nhits.
#     cells_zero_noice_nhits_lees_7 = FRBV(cells_zero_noice, "hit_id",truth_zero_noice_nhits_over_7["hit_id"]).drop("index",axis = 1).reset_index().drop("index",axis = 1)
#     #Removing all the data in cells that has over the 3 nhits.
#     cells_zero_noice_nhits_lees_7_over_3_with_weight_0 = FRBV(cells_zero_noice_nhits_lees_7,"hit_id",truth_zero_noice_over_3.hit_id).reset_index().drop("index",axis = 1)
#     #Making a list of hit_id that is not in truth but is in cells
#     hit_id_in_cells_but_not_in_truth = FRBV(cells_zero_noice_nhits_lees_7_over_3_with_weight_0,"hit_id",truth_zero_noice_nhits_lees_7_weight_0.hit_id).reset_index().drop("index",axis = 1)
#     #Using that cells hit_id not in truth, and then removing them from cells.
#     cells_zero_noice_nhits_lees_7_over_3_without_weight_0 = FRBV(cells_zero_noice_nhits_lees_7_over_3_with_weight_0,"hit_id",hit_id_in_cells_but_not_in_truth.hit_id).reset_index().drop("index",axis = 1)
    
    #Putting all the data from truth into hits:
    hits_merge = hits_zero_noice_nhits_lees_7_over_3_without_weight_0.merge(truth_zero_noice_nhits_lees_7_weight_0, how='left', on='hit_id')
    #Making a same layer filter 
    def same_layer_filter(hits_new):
        hits_long_layer_filtert = hits_new.drop_duplicates(subset = ["particle_id","volume_id", "layer_id"])
        return hits_long_layer_filtert
    hits_long_layer_filtert = same_layer_filter(hits_merge)
    #Making a function that can show the removed particles from same layer filter.
    def Remove_Elements(data, thrsshold):
        counted = Counter(data)
        temp_lst = []
        for i in counted:
            if counted[i] < thrsshold:
                temp_lst.append(i)
        res_lst = []
        for i in data:
            if i not in temp_lst:
                res_lst.append(i)
        data = [i for i in data if counted[i] >= thrsshold]
        return data , temp_lst , res_lst
    thrsshold = 4
    hits_filtert , particles_nhits_less_4 , particles_nhits_over_4 = Remove_Elements(hits_long_layer_filtert["particle_id"],thrsshold)
    #Removing the particles from the same layer filter in hits_merge and particle file
    hits_merge_filtered = FRBV(hits_long_layer_filtert ,"particle_id", particles_nhits_less_4).reset_index().drop("index",axis = 1)
    particles_merge_filtered = FRBV(particle_id_with_nhits_lees_7 ,"particle_id", particles_nhits_less_4).reset_index().drop("index",axis = 1)

    #Calculating the pt of a particle:
    Pt = []
    for i in (range(len(particles_merge_filtered))):
        func = (particles_merge_filtered.px[i]**2+particles_merge_filtered.py[i]**2)**(1/2)
        Pt.append([particles_merge_filtered.particle_id[i],func])
    #Making the pt filter loop
    Pt_1GeV = []
    for i in range(len(Pt)):
        if Pt[i][1] >= pt_cut_start and Pt[i][1] <= pt_cut_end: 
            Pt_1GeV.append(Pt[i])
    #Finding the particles from the pt filter 
    particles_over_Pt_1GeV = []
    for i in range(len(Pt_1GeV)):
        particles_over_Pt_1GeV.append(Pt_1GeV[i][0])
    # Making a function that keeps values that is in list, and removes the ones that is not. 
    def FCBV(file_name, column_name, list_of_values):
        return file_name[file_name[column_name].isin(list_of_values)]
    #Removing the rows from the pt filter.  
    particles_over_Pt_1GeV_filtered = FCBV(particles_merge_filtered ,"particle_id", particles_over_Pt_1GeV).reset_index().drop("index",axis = 1)
    hits_merge_over_Pt_1GeV_filtered = FCBV(hits_merge_filtered ,"particle_id", particles_over_Pt_1GeV).reset_index().drop("index",axis = 1)
    return  hits_merge_over_Pt_1GeV_filtered , particles_over_Pt_1GeV_filtered

In [7]:
%%time
hits_merge_all = []
for i in range(len(hits_all)):
    hits_merge_all.append(func_cleaning_data(hits_all[i],particles_all[i],truth_all[i],1,3))

print(len(hits_merge_all))
    
lenght_all_nodes_combined = []
for i in range(len(hits_merge_all)):
    lenght_all_nodes_combined.append(len(hits_merge_all[i][0]))
lenght_all_particles_combined = []
for i in range(len(hits_merge_all)):
    lenght_all_particles_combined.append(len(hits_merge_all[i][1]))
print("Amount of nodes:",sum(lenght_all_nodes_combined),"Amount of different particles:",sum(lenght_all_particles_combined))

100
Amount of nodes: 31623 Amount of different particles: 6457
Wall time: 1min 59s


### Cleaning data

In [8]:
def nodes_and_edges(data,x,y,z):
    def transform_and_merge(data,x,y,z):
        def car_to_cyl_cood(x,y,z):
            r = np.sqrt(x**2 + y**2)
            phi = np.arctan2(y,x)
            z = z
            return r , phi , z 
        r_hits , phi_hits , z_hits = car_to_cyl_cood(x,y,z)
        data["phi"] = phi_hits
        data["r"] = r_hits
        return data , r_hits , phi_hits , z_hits

    hits_merge , r_hits , phi_hits , z_hits = transform_and_merge(data,data.x , data.y , data.z)
    def car_to_sphe_cood(r,z):
        theta = np.arctan2(r,z)
        return theta

    theta = car_to_sphe_cood(r_hits,z_hits)
    eta = -np.log(np.tan(theta/2))
    rr = np.array(hits_merge.r)
    zz = np.array(hits_merge.z)
    pp = np.array(hits_merge.phi)
    dpp = []
    dpdr = []
    delta_phi = []
    delta_z = []
    delta_r = []
    delta_eta = []
    node_to_node_feat = []
    for i, idx in enumerate(range(len(pp))):
        for k, kdx in enumerate(range(len(rr))):
            dpdr = (pp[k]- pp[i])/(rr[k]-rr[i])
            dr = rr[k]-rr[i]
            z0 =  zz[i] - (rr[i]*(zz[k]-zz[i])/(rr[k]-rr[i]))
            if abs(dpdr) < 0.05 and abs(z0) < 249 and dr > 0:
                dpp.append([(hits_merge.particle_id[i] == hits_merge.particle_id[k])*1,dpdr, idx, kdx, hits_merge.hit_id[idx],hits_merge.hit_id[kdx],abs(dr),hits_merge.z[idx],hits_merge.z[kdx],hits_merge.r[idx],hits_merge.r[kdx],hits_merge.phi[idx],hits_merge.phi[kdx]])
                node_to_node_feat.append([hits_merge.z[idx],hits_merge.z[kdx],hits_merge.r[idx],hits_merge.r[kdx],hits_merge.phi[idx],hits_merge.phi[kdx]])

    dp_over_dr_TF = pd.DataFrame(dpp)
    dp_over_dr_TF.columns = ['Y_k' , "dpdr", 'node_1', 'node_2',"hit_id_node1","hit_id_node2","dr","z_node1","z_node2","r_node1","r_node2","phi_node1","phi_node2"]

    dp_over_dr_True = []
    for i in range(len(dpp)):
        if dpp[i][0] == 1:
            dp_over_dr_True.append(dpp[i])
    dp_over_dr_True = pd.DataFrame(dp_over_dr_True)
    dp_over_dr_True.columns = ['Y_k' , "dpdr" , 'node_1', 'node_2',"hit_id_node1","hit_id_node2","dr","z_node1","z_node2","r_node1","r_node2","phi_node1","phi_node2"]
    dp_over_dr_True = dp_over_dr_True.drop_duplicates(subset = "dpdr",ignore_index = True)

    dp_over_dr_False = []
    for i in range(len(dpp)):
        if dpp[i][0] == 0:
            dp_over_dr_False.append(dpp[i])
    dp_over_dr_False = pd.DataFrame(dp_over_dr_False)
    dp_over_dr_False.columns = ['Y_k' , "dpdr" , 'node_1', 'node_2',"hit_id_node1","hit_id_node2","dr","z_node1","z_node2","r_node1","r_node2","phi_node1","phi_node2"]
    dp_over_dr_False = dp_over_dr_False.drop_duplicates(subset = "dpdr",ignore_index = True)

    def edges_feats():
        for i, idx in enumerate(range(len(pp))):
            for k, kdx in enumerate(range(len(rr))):
                dpdr = (pp[k]- pp[i])/(rr[k]-rr[i])
                dr = rr[k]-rr[i]
                dphi = pp[k]- pp[i]
                dz = zz[k]-zz[i]
                deta = eta[k]-eta[i]
                z0 =  zz[i] - (rr[i]*(zz[k]-zz[i])/(rr[k]-rr[i]))
                if abs(dpdr) < 0.05 and abs(z0) < 249 and dr > 0:
                    delta_r.append([(hits_merge.particle_id[i] == hits_merge.particle_id[k])*1, abs(dr) , idx, kdx])
                    delta_phi.append([(hits_merge.particle_id[i] == hits_merge.particle_id[k])*1,abs(dphi), idx, kdx])
                    delta_z.append([(hits_merge.particle_id[i] == hits_merge.particle_id[k])*1,abs(dz), idx, kdx])
                    delta_eta.append([(hits_merge.particle_id[i] == hits_merge.particle_id[k])*1,abs(deta), idx, kdx])

        def only_true(data , x):
            liste_1 = pd.DataFrame(data)
            liste_1.columns = ['Y_k' , x, 'node_1', 'node_2']
            #liste_1 = liste_1.drop_duplicates(subset = [x],ignore_index = True)
            liste_2 = []
            for i in range(len(data)):
                if data[i][0] == 1:
                    liste_2.append(data[i])
            liste_2 = pd.DataFrame(liste_2)
            liste_2.columns = ['Y_k' , x , 'node_1', 'node_2']
            liste_2 = liste_2.drop_duplicates(subset = [x],ignore_index = True)
            return liste_1 , liste_2

        delta_r = only_true(delta_r,"dr")
        delta_phi = only_true(delta_phi,"dphi")
        delta_z = only_true(delta_z,"dz")
        delta_eta = only_true(delta_eta,"deta")

        def delta_R(delta_eta,delta_phi):
            delta_R = []    
            for i in range(len(delta_eta)):
                dR = (delta_eta.deta[i]**2 + delta_phi.dphi[i]**2)**(1/2)
                delta_R.append(dR)
            delta_R = pd.DataFrame(delta_R)
            delta_R.columns = ['delta_R']
            return delta_R

        return delta_r , delta_phi , delta_z , delta_eta

    #Making the datafram that contains the nodes ids and the nodes features. 
    nodes_id_plus_features = hits_merge[["hit_id","z","r","phi"]].copy()
    
    return nodes_id_plus_features , dp_over_dr_True , dp_over_dr_False

In [9]:
%%time

hits_merge_all = [nodes_and_edges(hits_merge_all[i][0] , hits_merge_all[i][0].x , hits_merge_all[i][0].y , hits_merge_all[i][0].z) for i in range(len(hits_merge_all))] 


C:\Users\nicol\AppData\Local\Temp/ipykernel_12372/2789436489.py:32: RuntimeWarning: invalid value encountered in double_scalars
  dpdr = (pp[k]- pp[i])/(rr[k]-rr[i])
C:\Users\nicol\AppData\Local\Temp/ipykernel_12372/2789436489.py:34: RuntimeWarning: invalid value encountered in double_scalars
  z0 =  zz[i] - (rr[i]*(zz[k]-zz[i])/(rr[k]-rr[i]))
C:\Users\nicol\AppData\Local\Temp/ipykernel_12372/2789436489.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  dpdr = (pp[k]- pp[i])/(rr[k]-rr[i])
C:\Users\nicol\AppData\Local\Temp/ipykernel_12372/2789436489.py:34: RuntimeWarning: divide by zero encountered in double_scalars
  z0 =  zz[i] - (rr[i]*(zz[k]-zz[i])/(rr[k]-rr[i]))


Wall time: 3min 11s


In [10]:
%%time
def REWMH(data):#removing_edges_with_to_many_hits_ids
    data_sorted_True = data[1].sort_values(by = ["hit_id_node1","dr"],ascending=True)
    data_True = data_sorted_True.drop_duplicates(subset = "hit_id_node1",ignore_index = True)
    data_sorted_False = data[2].sort_values(by = ["hit_id_node1","dr"],ascending=True)
    data_False = data_sorted_False.drop_duplicates(subset = "hit_id_node1",ignore_index = True)
    True_and_false = data_True.append(data_False, ignore_index=True)
    return True_and_false

def REWMHT(data):#removing_edges_with_to_many_hits_ids
    data_sorted_True = data[1].sort_values(by = ["hit_id_node1","dr"],ascending=True)
    data_True = data_sorted_True.drop_duplicates(subset = "hit_id_node1",ignore_index = True)
    return data_True

def REWMHF(data):#removing_edges_with_to_many_hits_ids
    data_sorted_False = data[2].sort_values(by = ["hit_id_node1","dr"],ascending=True)
    data_False = data_sorted_False.drop_duplicates(subset = "hit_id_node1",ignore_index = True)
    return data_False

All_edges_TF = [REWMH(hits_merge_all[i]) for i in range(len(hits_merge_all))] 
All_edges_T = [REWMHT(hits_merge_all[i]) for i in range(len(hits_merge_all))] 
All_edges_F = [REWMHF(hits_merge_all[i]) for i in range(len(hits_merge_all))] 


All_edges_TF_combined = pd.concat(All_edges_TF, ignore_index=True)
All_edges_T_combined = pd.concat(All_edges_T, ignore_index=True)
All_edges_F_combined = pd.concat(All_edges_F, ignore_index=True)

print(len(All_edges_TF_combined))
print(len(All_edges_T_combined))
print(len(All_edges_F_combined))


49838
24673
25165
Wall time: 1.66 s


In [17]:
%%time
from sklearn.model_selection import train_test_split
Y = All_edges_TF_combined.Y_k
X = All_edges_TF_combined[["z_node1","z_node2","r_node1","r_node2","phi_node1","phi_node2"]].copy()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=42)
print(Counter(Y_train))
display(len(X_train),len(X_test))

Counter({0: 22628, 1: 22226})


44854

4984

Wall time: 12 ms


# Begining to make a GNN, starting with MLPClassifier

In [20]:
%%time
model = MLPClassifier(hidden_layer_sizes = (200,100),random_state=1, max_iter = 300)
model.fit(X_train,Y_train)
prop_train = model.predict(X_train)
score_train = model.score(X_train,Y_train)
True_predicted_train = Counter(prop_train)[1]
False_predicted_train = Counter(prop_train)[0]

In [23]:
print(len(prop_train))
print("True_input vs True_output...:",Counter(Y_train)[1],"vs",True_predicted_test)
print("False_input vs False_output...:",Counter(Y_train)[0],"vs",False_predicted_test)
print(round(score_train,4)*100,"%")

44854
True_input vs True_output...: 22226 vs 2466
False_input vs False_output...: 22628 vs 2518
96.76 %


In [24]:
prop_test = model.predict(X_test)
score_test = model.score(X_test,Y_test)
True_predicted_test = Counter(prop_test)[1]
False_predicted_test = Counter(prop_test)[0]
print(len(prop_test))
print("True_input vs True_output...:",Counter(Y_test)[1],"vs",True_predicted_test)
print("False_input vs False_output...:",Counter(Y_test)[0],"vs",False_predicted_test)
print(round(score_test,4)*100,"%")

4984
True_input vs True_output...: 2447 vs 2466
False_input vs False_output...: 2537 vs 2518
96.41 %


In [ ]:
X_true = []
for i in range(len(prob1)):
    if prob1[i] == 1:
        X_true.append(X1[i])


edges = np.array(X_true)
xyz = edges[:,2:8]
plt.figure(69, figsize= (10,10))
for i in range(len(xyz)):
    px = np.array([xyz[i,0],xyz[i,1]])
    py = np.array([xyz[i,4], xyz[i,5]])
    plt.scatter(px,py, s = 10, color = 'black')
    plt.plot(px,py, '--', alpha = 0.1, color = 'b')
    #plt.xlim(35,100)
    #plt.ylim(15,80)

### Kigger på data

### Ploting data pre GNN

In [ ]:
def plotting_data(data_rough,data_clean):
    XYZ_rough = [[],[],[]]
    XYZ_clean = [[],[],[]]
    for i, idx in enumerate(range(len(data_rough.x))):
        if abs(data_rough.z[i]) < 2000:
            XYZ_rough[0].append((data_rough.x[idx]))
            XYZ_rough[1].append((data_rough.y[idx]))
            XYZ_rough[2].append((data_rough.z[idx]))
    print(len(XYZ_rough[0]), len(XYZ_rough[1]), len(XYZ_rough[2]))

    for i, idx in enumerate(range(len(data_clean.x))):
        if abs(data_clean.z[i]) < 2000:
            XYZ_clean[0].append((data_clean.x[idx]))
            XYZ_clean[1].append((data_clean.y[idx]))
            XYZ_clean[2].append((data_clean.z[idx]))
    print(len(XYZ_clean[0]), len(XYZ_clean[1]), len(XYZ_clean[2]))
    return XYZ_rough , XYZ_clean

XYZ_rough , XYZ_clean = plotting_data(hits_train_100,hits_merge)

In [ ]:
zoom = 2000
figsize = 5
alpha = 0.35

fig = plt.figure(1, figsize = (figsize,figsize))
ax = plt.axes(projection='3d')
ax.scatter3D(XYZ_rough[0],XYZ_rough[1],XYZ_rough[2], c = XYZ_rough[0], alpha = alpha)
plt.xlim(-zoom,zoom)
plt.ylim(-zoom,zoom)
ax.set_zlim(-zoom,zoom)


fig = plt.figure(2, figsize = (figsize,figsize))
ax = plt.axes(projection='3d')
ax.scatter3D(XYZ_clean[0],XYZ_clean[1],XYZ_clean[2], c = XYZ_clean[0] , alpha = alpha)
plt.xlim(-zoom,zoom)
plt.ylim(-zoom,zoom)
ax.set_zlim(-zoom,zoom)

### Looking at the detector

In [ ]:
x = data_detectors.cx
y = data_detectors.cy
z = data_detectors.cz
fig = plt.figure(5, figsize = (10,10))
ax = plt.axes(projection='3d')
ax.scatter3D(z , y ,x, c = y, alpha = 0.5)